In [1]:
import sys
sys.path.append('/home/nigitha/ros2_ws_rnd/src')
import subprocess

In [2]:
! source /opt/ros/humble/setup.bash


In [3]:
import rosbag2_py
import numpy as np
import rclpy
from rclpy.node import Node
from nav_msgs.msg import OccupancyGrid, Path
from geometry_msgs.msg import Twist, PoseStamped
from rclpy.serialization import deserialize_message
import os
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from nav_msgs.msg import OccupancyGrid
import subprocess
import pandas as pd


In [39]:
class ExtractPath():
    def __init__(self, bag_file):
        
        self.bag_file = bag_file       

        # Containers for data
        self.path = None
        self.map = None
        self.map_resolution = None
        self.map_origin = None

        # Process messages
        self.process_bag(bag_file)

    def process_bag(self, bag_file):
        reader = rosbag2_py.SequentialReader()
        storage_options = rosbag2_py.StorageOptions(uri=bag_file, storage_id='sqlite3')
        converter_options = rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')        
        reader.open(storage_options, converter_options)
        while reader.has_next():
            topic, msg, t = reader.read_next()

            if topic == "/robot_path":
                self.process_path(msg, t)
            elif topic == "/map":
                self.process_map(msg, t)
            elif topic == "/goal_pose":
                self.process_goal(msg, t)
                
            elif topic == "/astar":
                self.process_waypoints(msg, t)

    def process_path(self, msg, timestamp):
        path_msg = deserialize_message(msg, Path)
        self.path = path_msg
        self.end_time = timestamp
        
    def process_waypoints(self, msg, timestamp):
        path_msg = deserialize_message(msg, Path)
        self.waypoints = path_msg
        
    def process_goal(self, msg, timestamp):
        goal_msg = deserialize_message(msg, PoseStamped)
        self.goal = [goal_msg.pose.position.x, goal_msg.pose.position.y]
        self.start_time = timestamp

    def process_map(self, msg, timestamp):
        map_msg = deserialize_message(msg, OccupancyGrid)
        self.map_received = True
        self.map_data = map_msg
        # self.save_map()
        # Extract metadata
        self.map_resolution = map_msg.info.resolution
        width = map_msg.info.width
        height = map_msg.info.height
        self.map_origin = [map_msg.info.origin.position.x, map_msg.info.origin.position.y]

        # Extract the occupancy data
        self.map = np.array(map_msg.data).reshape((height, width))
        
        
    


In [42]:
def extract_coordinates(path_msg):
    """Extract X, Y coordinates from the path message."""    
    x_coords = []
    y_coords = []
    x_prev = None
    y_prev =  None
    t_prev = None
    for i, pose in enumerate(path_msg.poses):
        x_c = pose.pose.position.x
        y_c = pose.pose.position.y
        t_c = pose.header.stamp.sec
        if x_prev is not None: 
            dist = ((x_c-x_prev)**2 + (y_c-y_prev)**2)**0.5
            dur = t_c - t_prev
            
            # if dist>0.05:
            if dur> 0.5:
                x_coords.append(x_c)
                y_coords.append(y_c)
                x_prev = x_c
                y_prev = y_c
                t_prev = t_c
        else:
            x_coords.append(x_c)
            y_coords.append(y_c)            
            x_prev = x_c
            y_prev = y_c
            t_prev = t_c
    
    return np.array(x_coords), np.array(y_coords)

In [51]:
def extract_wp_coordinates(path_msg):
    """Extract X, Y coordinates from the path message."""    
    x_coords = []
    y_coords = []
    for i, pose in enumerate(path_msg.poses):
        x_c = pose.pose.position.x
        y_c = pose.pose.position.y
        x_coords.append(x_c)
        y_coords.append(y_c)
    
    return np.array(x_coords), np.array(y_coords)

In [52]:
import os

file_path = '/home/nigitha/ros2_ws_rnd/src/Evaluation/il_01'
extractor = ExtractPath(file_path)
duration = extractor.end_time-extractor.start_time
duration = duration/ 1e+09
map_=extractor.map
map_origin_ = extractor.map_origin
map_resolution_ = extractor.map_resolution
path = {'path': extract_coordinates(extractor.path), 
        'waypoints': extract_wp_coordinates(extractor.waypoints),'map':map_, 
        'map_origin':map_origin_, 'map_resolution':map_resolution_,
        'duration':duration, 'goal':extractor.goal}  

print(path['waypoints'])




[INFO] [1739546052.707789620] [rosbag2_storage]: Opened database '/home/nigitha/ros2_ws_rnd/src/Evaluation/il_01/il_01_0.db3' for READ_ONLY.


(array([-13.04999935, -12.54999935, -12.04999934, -11.54999933,
       -11.04999932, -10.54999932, -10.04999931,  -9.5499993 ,
        -9.04999929,  -8.54999929,  -8.04999928,  -7.54999927,
        -7.04999926,  -6.54999926,  -6.04999925,  -5.54999924,
        -5.04999923,  -4.79999923,  -4.39999922,  -3.89999922,
        -3.39999921]), array([-1.7999999 , -2.0499999 , -2.54999991, -3.04999992, -3.39999992,
       -3.49999993, -3.49999993, -3.74999993, -3.74999993, -3.74999993,
       -3.89999993, -3.94999993, -4.04999993, -4.04999993, -3.64999993,
       -3.14999992, -2.99999992, -2.59999991, -2.09999991, -1.7999999 ,
       -1.34999989]))


In [71]:
import plotly.graph_objects as go
import numpy as np
from scipy.ndimage import zoom

def transform_path_to_map_pixel(path, map_origin, new_resolution):
    map_x = (path[0] - map_origin[0]) / new_resolution
    map_y = (path[1] - map_origin[1]) / new_resolution
    return map_x.astype(int), map_y.astype(int)

def plot_path_plotly(path, resolution_factor=3):
    bright_colors = [
        "#0000FF",  "#FF8C00", "#FF00FF", 
        "#00FFFF", "#FF1493", "#8A2BE2", "#FFD700", "#00FA9A"
    ]
        
    fig = go.Figure()
    map_data = path['map']
    map_resolution = path['map_resolution']
    map_origin = path['map_origin']
    goal = np.array(path['goal'])
    
   
    color = "#FF8C00"

    map_resolution /= resolution_factor  
    map_data = zoom(map_data, (resolution_factor, resolution_factor), order=1)
    map_x, map_y = transform_path_to_map_pixel(path['path'], map_origin, map_resolution)
    map_wp_x, map_wp_y = transform_path_to_map_pixel(path['waypoints'], map_origin, map_resolution)
     
    # Add map as a heatmap (black and white)
    fig.add_trace(go.Heatmap(
        z=map_data, 
        colorscale='blackbody', 
        opacity=0.6,  # Set opacity for better visualization
        zmin=0, zmax=np.max(map_data),
        showscale=False 
    ))

                
                
    fig.add_trace(go.Scatter(
        x=[map_x[0]], y=[map_y[0]],
        mode='markers', marker=dict(color='#00FF00', size=10),
        name='Start Position'
    ))


    # Plot path (legend only for the first run of each model)
    fig.add_trace(go.Scatter(
        x=map_x, y=map_y, 
        mode='lines',
        line=dict(color=color, width=2),
        name='robot path'
    ))
    # Plot path (legend only for the first run of each model)
    fig.add_trace(go.Scatter(
        x=map_wp_x, y=map_wp_y, 
        mode='markers',
        marker=dict(color='#FF1493', size=5),
        name='way points'
    ))
            

            
    # Convert goal to pixel coordinates and plot
    goal_x, goal_y = transform_path_to_map_pixel(goal, map_origin, map_resolution)
    fig.add_trace(go.Scatter(
        x=[goal_x], y=[goal_y],
        mode='markers', marker=dict(color='red', size=10),
        name='Goal Position'
    ))
    
    # Adjust layout to match the image aspect and set the figure size
    fig.update_layout(
        title="Multiple Robot Paths on Map (Interactive)",
        xaxis=dict(title="Map X (pixels)", scaleanchor="y"),
        yaxis=dict(title="Map Y (pixels)", scaleanchor="x"),
        showlegend=True,
        autosize=False,
        width=1000,  # Set width of the figure
        height=800,  # Set height of the figure
    )
    fig.update_layout(
    legend=dict(
        x=1,  # Position legend to the right of the plot
        y=1,  # Position it at the top
        xanchor='right',  # Anchor legend to the left of its position
        yanchor='top',  # Anchor legend to the top of its position
        bgcolor='rgba(255, 255, 255, 0.9)',  # Optional: Set background color of legend
        bordercolor='Black',  # Optional: Set border color
        borderwidth=1 , # Optional: Set border width
        font=dict(
            size=20,  # Increase font size
        ),
        itemwidth=40,  # Increase size of the legend markers (lines)
        traceorder='normal' 
    )
)

    fig.show()

# Example usage:
# plot_multiple_paths_plotly(paths['scenario01'])


In [72]:
plot_path_plotly(path)

In [35]:
path['waypoints']

(array([-13.04999935]), array([-1.7999999]))

In [37]:
def get_dist_from_goal(path, goal):
    return np.linalg.norm([path[0][-1],path[1][-1]]- np.array(goal))

def get_path_len(path):
    path_length = 0
    for i in range(1, len(path[0])):
        delta = np.linalg.norm(np.array([path[0][i],path[1][i]]) - np.array([path[0][i-1],path[1][i-1]]))
        path_length += delta
    return path_length

In [38]:
def get_metrics(runs, goal_reach_thresh=0.5):
    
    total_distance_from_goal = []
    total_path_length= []
    success_count = 0
    total_duration = []
    total_start_to_goal_dist = []
    for run_name, run in runs.items(): 
        path =  run['path']     
        goal = run['goal']
        path_length = get_path_len(path)
        
        distance_from_goal = get_dist_from_goal(path, goal)  
        total_distance_from_goal.append(distance_from_goal)
        
        if distance_from_goal < goal_reach_thresh:  # Threshold for success
            success_count += 1
            total_path_length.append(path_length)
            total_duration.append(run['duration'])
            
        start_to_goal_dist = np.linalg.norm(np.array([path[0][0],path[1][0]]) - np.array(goal))
        total_start_to_goal_dist.append(start_to_goal_dist)
        
    num_paths = len(runs)
    metrics = {
        'success_rate': success_count / num_paths,
        'mean_distance_from_goal': np.mean(total_distance_from_goal),
        'std_distance_from_goal': np.std(total_distance_from_goal),
        'mean_path_length': np.mean(total_path_length),
        'std_path_length': np.std(total_path_length),
        'mean_duration': np.mean(total_duration),
        'std_duration': np.std(total_duration),
        'mean_start_to_goal_dist': np.mean(total_start_to_goal_dist),
        'std_start_to_goal_dist': np.std(total_start_to_goal_dist)
    }

    print(metrics)
    return metrics

In [39]:
metric_dict = {}
for traj_name, traj_paths in paths.items():
    print(traj_name)
    metric_dict[traj_name] = {}
    for modal_name, modal_paths  in traj_paths.items():
        print(modal_name)
        metric_dict[traj_name][modal_name]= get_metrics(modal_paths)

scenario03
image
{'success_rate': 0.2, 'mean_distance_from_goal': 2.086303033226049, 'std_distance_from_goal': 0.9342759494880284, 'mean_path_length': 3.6421089528945725, 'std_path_length': 0.0, 'mean_duration': 103.670016694, 'std_duration': 0.0, 'mean_start_to_goal_dist': 3.580134875918489, 'std_start_to_goal_dist': 0.009544893228712225}
laser_image
{'success_rate': 0.8, 'mean_distance_from_goal': 0.6537745973503855, 'std_distance_from_goal': 0.7699651915919005, 'mean_path_length': 3.8572094167128697, 'std_path_length': 0.23397254518384297, 'mean_duration': 171.43452788300002, 'std_duration': 22.57489963811333, 'mean_start_to_goal_dist': 3.5763976590176405, 'std_start_to_goal_dist': 0.01209367863520614}
laser
{'success_rate': 0.4, 'mean_distance_from_goal': 1.114191032672439, 'std_distance_from_goal': 0.7208364644428434, 'mean_path_length': 3.8733554698712647, 'std_path_length': 0.09820524955499699, 'mean_duration': 214.3012840545, 'std_duration': 52.12225248649999, 'mean_start_to_go

/home/nigitha/ros2_rnd_venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/home/nigitha/ros2_rnd_venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide

/home/nigitha/ros2_rnd_venv/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/nigitha/ros2_rnd_venv/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in divide

/home/nigitha/ros2_rnd_venv/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning:

invalid value encountered in scalar divide



In [40]:
final_metrics_df = pd.DataFrame.from_dict(
    {(outer_key, inner_key): values for outer_key, inner_dict in metric_dict.items() for inner_key, values in inner_dict.items()},
    orient='index'
).reset_index()

# Rename columns
final_metrics_df.columns = ["Scenario", "Model"] + list(final_metrics_df.columns[2:])

In [41]:
final_metrics_df

,Scenario,Model,success_rate,mean_distance_from_goal,std_distance_from_goal,mean_path_length,std_path_length,mean_duration,std_duration,mean_start_to_goal_dist,std_start_to_goal_dist
0,scenario03,image,0.2,2.086303,0.934276,3.642109,0.000000,103.670017,0.000000,3.580135,0.009545
1,scenario03,laser_image,0.8,0.653775,0.769965,3.857209,0.233973,171.434528,22.574900,3.576398,0.012094
2,scenario03,laser,0.4,1.114191,0.720836,3.873355,0.098205,214.301284,52.122252,3.609669,0.046874
3,scenario03,potential_field,0.0,3.149049,0.125836,NaN,NaN,NaN,NaN,3.571604,0.010032
4,scenario01,image,0.2,1.214001,0.830300,3.734975,0.000000,397.885342,0.000000,3.352660,0.067997
5,scenario01,laser_image,0.8,0.743976,1.152753,3.855920,0.126533,107.440918,10.542250,3.474438,0.030488
6,scenario01,laser,1.0,0.139229,0.037228,3.983443,0.218685,118.864111,12.899741,3.422277,0.113051
7,scenario01,potential_field,1.0,0.030399,0.011430,4.583655,0.073567,144.010992,9.297655,3.476483,0.042747
8,scenario04,image,0.0,2.296506,0.067052,NaN,NaN,NaN,NaN,3.236528,0.055388
9,scenario04,laser_image,0.0,2.235196,0.095966,NaN,NaN,NaN,NaN,3.160874,0.041343


In [19]:
def get_individual_metrics(runs, goal_reach_thresh=0.5):
    
  
    metrics = {}
    for run_name, run in runs.items(): 
        path =  run['path']     
        goal = run['goal']
        path_length = get_path_len(path)
        
        
        distance_from_goal = get_dist_from_goal(path, goal)   
        
        if distance_from_goal < goal_reach_thresh:  # Threshold for success
            success =1
            
        else:
            success = 0
            
        start_to_goal_dist = np.linalg.norm(np.array([path[0][0],path[1][0]]) - np.array(goal))
        metrics[run_name] = {'success':success, 'path_length':path_length, 
                             'distance_from_goal':distance_from_goal, 'duration':run['duration'],
                             'start_to_goal_dist':start_to_goal_dist}

    return metrics

In [23]:
print('scenario04 - laser')
metrics = get_individual_metrics(paths['scenario04']['laser'])
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df

scenario04 - laser


,run02,run05,run04,run01,run03,run06
success,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
path_length,0.926200,1.068488,1.246997,3.274661,0.530070,3.071116
distance_from_goal,2.301276,2.113679,1.970813,0.122861,2.647197,0.164909
duration,35.948536,36.243582,35.859142,121.556366,23.291377,87.162517
start_to_goal_dist,3.154356,3.165689,3.157786,3.183799,3.163647,3.188314


In [29]:
print('scenario01 - laser')
metrics = get_individual_metrics(paths['scenario01']['laser'])
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df

scenario01 - laser


,run02,run03,run04,run01,run05
success,1.000000,1.000000,1.000000,1.000000,1.000000
path_length,3.886467,4.032829,4.254475,3.617899,4.125543
distance_from_goal,0.160950,0.139275,0.158145,0.170231,0.067544
duration,119.276477,124.832506,118.890174,95.966786,135.354613
start_to_goal_dist,3.444670,3.445535,3.488814,3.204720,3.527646


In [59]:
print('scenario01 - image')
metrics = get_individual_metrics(paths['scenario01']['image'])
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df

scenario01 - image


,run01,run04,run03,run02,run05
success,0.000000,0.000000,0.000000,0.000000,1.000000
path_length,4.491283,0.613465,4.559268,4.891256,3.734975
distance_from_goal,0.568204,2.749843,1.175644,1.179930,0.396385
duration,292.329102,28.740346,254.301427,373.452430,397.885342
start_to_goal_dist,3.423884,3.353336,3.378660,3.383007,3.224411


In [60]:
print('scenario01 - laser_image')
metrics = get_individual_metrics(paths['scenario01']['laser_image'])
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df

scenario01 - laser_image


,run04,run05,run02,run03,run01
success,1.000000,1.000000,0.000000,1.000000,1.000000
path_length,3.663931,3.997385,0.497679,3.826892,3.935470
distance_from_goal,0.214454,0.150131,3.047760,0.213107,0.094431
duration,112.232658,93.426185,25.985142,121.608848,102.495980
start_to_goal_dist,3.512449,3.439083,3.466995,3.507541,3.446124
